# AI Search02 - Agentic retrieval using Azure AI Search and Azure AI Agent Service

Use this notebook to create an agentic retrieval pipeline built on Azure AI Search and an Azure AI Agent.

In this walkthrough, you will:

+ Create an "earth_at_night" search index
+ Load it with documents from a GitHub URL
+ Create a knowledge agent on Azure AI Search that points to an LLM for intelligent query planning
+ Create a Foundry agent in Azure AI Foundry to determine when queries are needed
+ Create a Azure AI Agent tool (client) to orchestrate all requests
+ Start a chat with the agent

This notebook is referenced in [Build an agentic retrieval pipeline in Azure AI Search](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-pipeline).

This exercise differs from the [Agentic Retrieval Quickstart](https://learn.microsoft.com/azure/search/search-get-started-agentic-retrieval) in how it uses Azure AI Agent to determine whether to retrieve data from the index, and how it uses an agent tool for orchestration.

## Prerequisites

+ Azure AI Search, basic tier or higher, in any region that supports semantic ranker.

+ Azure OpenAI, and you should have an **Azure AI Developer** role assignment to create a Foundry project.

+ An [Azure AI agent and Foundry project](https://learn.microsoft.com/azure/ai-services/agents/quickstart?pivots=ai-foundry-portal), created in the Azure AI Foundry portal, with the basic setup, used for creating the Foundry agent.

+ A deployment of a [supported model](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-create#supported-models) in your Foundry project. This notebook uses gpt-4o-mini. We recommend 100,000 token capacity. You can find capacity and the rate limit in the model deployments list in the Azure AI Foundry portal.

We recommend creating a virtual environment to run this sample code. In Visual Studio Code, open the control palette (ctrl-shift-p) to create an environment. This notebook was tested on Python 3.10.

## Set up connections

 `.env` modify the environment variables to use your Azure endpoints. You need endpoints for:

+ Azure AI Search
+ Azure OpenAI
+ Azure AI Foundry project

You can find endpoints for Azure AI Search and Azure OpenAI in the [Azure portal](https://portal.azure.com).

You can find the project endpoint in the [Azure AI Foundry portal](https://ai.azure.com).
   A hypothetical endpoint might look like this: `https://your-foundry-resource.services.ai.azure.com/api/projects/your-foundry-project`

[Checkpoint 3]
![alt text](Image\image3.png)

## Load Connections

Load the environment variables to set up connections and object names.

In [30]:
from dotenv import load_dotenv  
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

# Load environment variables from .env file (override=True ensures .env values take precedence)
load_dotenv(override=True) 

# Define default deployment names for Azure OpenAI and Azure Search index
AZURE_OPENAI_GPT_DEPLOYMENT="gpt-4.1-mini"
AZURE_SEARCH_INDEX_NAME="12-earth-at-night"
project_endpoint = os.environ["PROJECT_ENDPOINT"]
agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]

# Authentication setup using Azure's default credential chain
# This will try managed identity, Azure CLI, Visual Studio, etc. in order
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")

# Search index name - note the default differs from the constant above
index_name = os.getenv("AZURE_SEARCH_INDEX", "12-earth-at-night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]  # Your Azure OpenAI service endpoint
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")

# Embedding model deployment settings for vector search
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")

# Agent name for both Azure Search knowledge agent and AI Foundry agent
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "12-earth-search-agent")

## Create search index on Azure AI Search

This steps create a search index that contains plain text and vector content. You can use any existing search index, but it must meet the [criteria for agentic retrieval workloads](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-index). The primary schmea requirement is that is has a semantic configuration, with a `default_configuration_name`.

In [31]:
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField
from azure.search.documents.indexes import SearchIndexClient

# Create a comprehensive search index that supports both text and vector search
# This index is designed for agentic retrieval workloads with semantic capabilities
index = SearchIndex(
    name=index_name,  # Use the index name from environment configuration
    fields=[
        # Primary key field - required for all Azure Search indexes
        SearchField(name="id", type="Edm.String", key=True, filterable=True, sortable=True, facetable=True),
        
        # Text content field containing the actual document chunks
        # This field is searchable but not filterable/sortable to optimize for text search
        SearchField(name="page_chunk", type="Edm.String", filterable=False, sortable=False, facetable=False),
        
        # Vector embedding field for semantic similarity search
        # - Collection(Edm.Single): Array of floating-point numbers representing the embedding
        # - stored=False: Don't store the raw embeddings (saves space, still searchable)
        # - vector_search_dimensions=3072: Dimension size for text-embedding-3-large model
        # - vector_search_profile_name: Links to the vector search configuration below
        SearchField(name="page_embedding_text_3_large", type="Collection(Edm.Single)", stored=False, vector_search_dimensions=3072, vector_search_profile_name="hnsw_text_3_large"),
        
        # Page number field for filtering and sorting results by document location
        SearchField(name="page_number", type="Edm.Int32", filterable=True, sortable=True, facetable=True)
    ],
    
    # Vector search configuration using HNSW (Hierarchical Navigable Small World) algorithm
    vector_search=VectorSearch(
        # Vector search profiles define how vector fields are searched
        profiles=[VectorSearchProfile(
            name="hnsw_text_3_large",  # Profile name referenced by vector fields
            algorithm_configuration_name="alg",  # Links to algorithm configuration
            vectorizer_name="azure_openai_text_3_large"  # Links to vectorizer configuration
        )],
        
        # HNSW algorithm configuration for approximate nearest neighbor search
        # HNSW provides good balance between search speed and accuracy
        algorithms=[HnswAlgorithmConfiguration(name="alg")],
        
        # Vectorizer configuration - defines how text is converted to embeddings
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="azure_openai_text_3_large",
                parameters=AzureOpenAIVectorizerParameters(
                    resource_url=azure_openai_endpoint,  # Your Azure OpenAI endpoint
                    deployment_name=azure_openai_embedding_deployment,  # Embedding model deployment
                    model_name=azure_openai_embedding_model  # Embedding model name
                )
            )
        ]
    ),
    
    # Semantic search configuration for intelligent ranking and understanding
    # This is required for agentic retrieval workloads
    semantic_search=SemanticSearch(
        default_configuration_name="semantic_config",  # Default semantic configuration
        configurations=[
            SemanticConfiguration(
                name="semantic_config",
                # Define which fields are prioritized for semantic understanding
                prioritized_fields=SemanticPrioritizedFields(
                    # Content fields contain the main text content for semantic analysis
                    content_fields=[
                        SemanticField(field_name="page_chunk")  # Use our text content field
                    ]
                    # Note: We could also define title_fields and keyword_fields for richer semantic understanding
                )
            )
        ]
    )
)

# Create the search index client and deploy the index
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

# Create or update the index (idempotent operation)
# If index exists, it will be updated with new schema; if not, it will be created
index_client.create_or_update_index(index)
print(f"Index '{index_name}' created or updated successfully")

Index '12-earth-at-night' created or updated successfully


[Checkpoint 4]
![alt text](Image\image4.png)

## Upload sample documents

This sample uses data from NASA's Earth at Night e-book. It's retrieved from the sample data GitHub repository and passed to the search client for indexing.

In [32]:
import requests
from azure.search.documents import SearchIndexingBufferedSender

# URL pointing to pre-processed NASA Earth at Night e-book data
# This data is already chunked and formatted for search indexing
url = "https://raw.githubusercontent.com/Azure-Samples/azure-search-sample-data/refs/heads/main/nasa-e-book/earth-at-night-json/documents.json"

# Download the sample documents from GitHub
# The documents are already in the correct JSON format with fields matching our index schema
documents = requests.get(url).json()

# Use SearchIndexingBufferedSender for efficient batch uploading
# This class automatically handles batching, retries, and error handling
# The 'with' statement ensures proper resource cleanup
with SearchIndexingBufferedSender(
    endpoint=endpoint,           # Azure Search service endpoint
    index_name=index_name,      # Target index name
    credential=credential       # Authentication credentials
) as client:
    # Upload all documents in batches
    # The sender will automatically:
    # - Split documents into optimal batch sizes
    # - Generate embeddings using the configured vectorizer
    # - Handle retries for failed uploads
    # - Provide status updates
    client.upload_documents(documents=documents)

print(f"Documents uploaded to index '{index_name}'")

Documents uploaded to index '12-earth-at-night'


## Create a knowledge agent on Azure AI Search

This steps creates a knowledge agent on Azure AI Search. This agent is a wrapper to a large language model, used for sending queries to an agentic retrieval pipeline. The maximum output size refers to the query response. Setting this value helps you control token usage and how many tokens are sent to the LLM.

In [33]:
# Import classes for creating and managing Azure Search knowledge agents
from azure.search.documents.indexes.models import KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex, KnowledgeAgentRequestLimits, AzureOpenAIVectorizerParameters
from azure.search.documents.indexes import SearchIndexClient

# Create a knowledge agent - an AI-powered search orchestrator
# The knowledge agent acts as an intelligent query planner that:
# - Analyzes user queries for intent and complexity
# - Decides what search strategies to use (keyword, vector, semantic, hybrid)
# - Plans multi-step retrieval when needed
# - Formats and ranks results for optimal relevance
agent = KnowledgeAgent(
    name=agent_name,  # Unique name for this knowledge agent
    
    # Configure the LLM models that power the agent's intelligence
    models=[
        KnowledgeAgentAzureOpenAIModel(
            # Connect to your Azure OpenAI GPT deployment
            azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                resource_url=azure_openai_endpoint,        # Your Azure OpenAI service endpoint
                deployment_name=azure_openai_gpt_deployment, # GPT model deployment name
                model_name=azure_openai_gpt_model           # GPT model name (e.g., gpt-4.1-mini)
            )
        )
    ],
    
    # Define which search indexes this agent can query
    target_indexes=[
        KnowledgeAgentTargetIndex(
            index_name=index_name,          # The search index we created earlier
            default_reranker_threshold=2.5  # Minimum relevance score for including results
                                           # Higher values = more selective results
                                           # Lower values = more comprehensive results
        )
    ],
    
    # Set limits to control resource usage and response quality
    request_limits=KnowledgeAgentRequestLimits(
        max_output_size=10000  # Maximum number of tokens in agent responses
                              # Helps control costs and ensures responses fit in context windows
    )
)

# Create the search index client and deploy the knowledge agent
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

# Create or update the knowledge agent (idempotent operation)
# The agent becomes available immediately after creation
index_client.create_or_update_agent(agent)
print(f"Knowledge agent '{agent_name}' created or updated successfully")

Knowledge agent '12-earth-search-agent' created or updated successfully


In [34]:
# Verify that the knowledge agent was created successfully and is accessible
# This is important because agent creation might succeed but the agent might not be immediately available
try:
    # Retrieve agent information from Azure Search
    agent_info = index_client.get_agent(agent_name)
    print(f"Knowledge agent '{agent_info.name}' is available and working.")
except Exception as e:
    print(f"Knowledge agent '{agent_name}' is not available or not working: {e}")

Knowledge agent '12-earth-search-agent' is available and working.


## Create an Azure AI Agent

In the Azure AI Foundry, an agent is a smart micro-service that can do RAG. The purpose of this specific agent is to decide when to send a query to the agentic retrieval pipeline.

In [35]:
# Import the Azure AI Projects client for managing AI agents and workflows
from azure.ai.projects import AIProjectClient

# Create a client for the Azure AI Foundry project
project_client = AIProjectClient(
    endpoint=project_endpoint,  # Your AI Foundry project endpoint
    credential=credential       # Use the same Azure credential for authentication
)

# List all existing agents in the project (for debugging/verification)
list(project_client.agents.list_agents())

[{'id': 'asst_a6cWPZOs27r61GKiWi671INY', 'object': 'assistant', 'created_at': 1753014478, 'name': '13-earth-search-agent', 'description': None, 'model': 'gpt-4.1-mini', 'instructions': '\nA Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'},
 {'id': 'asst_hHwJMSNzJLg0YggUmhEZW744', 'object': 'assistant', 'created_at': 1753013020, 'name': '12-earth-search-agent', 'description': None, 'model': 'gpt-4.1-mini', 'instructions': '\nA Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 

In [36]:
# Define the system instructions for the AI agent
instructions = """
A Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].
If you do not have the answer, respond with "I don't know".
"""
# Key aspects of these instructions:
# - Domain-specific: Focused on Earth at night topics
# - Citation requirement: Must reference sources with [ref_id] format
# - Honest about limitations: Says "I don't know" when information isn't available
# - Structured responses: Expects JSON-formatted source data

# Create the AI agent in Azure AI Foundry
# This agent will orchestrate the entire RAG (Retrieval-Augmented Generation) pipeline
agent = project_client.agents.create_agent(
    model=agent_model,      # The LLM model to use (e.g., gpt-4.1-mini)
    name=agent_name,        # Agent name (same as knowledge agent for consistency)
    instructions=instructions  # System prompt that defines agent behavior
)
# The agent will:
# - Decide when to retrieve information from the search index
# - Process user queries intelligently
# - Coordinate with tools (like our agentic retrieval function)
# - Format responses according to the instructions

print(f"AI agent '{agent_name}' created or updated successfully")

AI agent '12-earth-search-agent' created or updated successfully


[Checkpoint 5]
![alt text](Image\image5.png)

In [37]:
# Verify that the AI agent was created successfully in the Foundry project
# This ensures the agent is ready to handle conversations and tool calls
try:
    # Retrieve agent information using the agent ID returned from creation
    agent_info = project_client.agents.get_agent(agent.id)
    print(f"AI agent '{agent_info.name}' is available and working.")
except Exception as e:
    print(f"AI agent '{agent_name}' is not available or not working: {e}")

AI agent '12-earth-search-agent' is available and working.


## Add an agentic retrieval tool to AI Agent

An end-to-end pipeline needs an orchestration mechanism for coordinating calls to the retriever and agent. The pattern described in this notebook uses a [tool](https://learn.microsoft.com/azure/ai-services/agents/how-to/tools/function-calling) for this task. The tool calls the Azure AI Search knowledge retrieval client and the Azure AI agent, and it drives the conversations with the user.

In [38]:
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams

# Create a knowledge retrieval client that connects to our Azure Search knowledge agent
# This client handles the communication between the AI agent and the search index
agent_client = KnowledgeAgentRetrievalClient(
    endpoint=endpoint,          # Azure Search service endpoint
    agent_name=agent_name,      # Name of the knowledge agent we created
    credential=credential       # Authentication credentials
)

# Create a conversation thread for managing multi-turn conversations
# Threads maintain context across multiple exchanges with the agent
thread = project_client.agents.threads.create()

# Dictionary to store retrieval results mapped to message IDs
# This allows us to access detailed retrieval information later for analysis
retrieval_results = {}

def agentic_retrieval() -> str:
    """
    Agentic retrieval function that searches NASA e-book data about Earth at night.
    
    This function serves as a "tool" that the AI agent can call when it needs to
    retrieve information from the search index. The agent decides autonomously
    when to use this tool based on the user's query.
    
    Returns:
        str: JSON-formatted search results with reference IDs for citation
    
    The function performs these steps:
    1. Gets recent conversation history for context
    2. Sends the conversation to the knowledge agent for intelligent retrieval
    3. Stores detailed retrieval results for later analysis
    4. Returns the formatted response to the AI agent
    """
    
    # Retrieve the last 5 messages from the conversation thread
    # This provides context for the knowledge agent to understand the query intent
    # DESCENDING order gets the most recent messages first
    messages = project_client.agents.messages.list(
        thread.id, 
        limit=5, 
        order=ListSortOrder.DESCENDING
    )
    
    # Convert iterator to list and reverse to get chronological order
    # (oldest message first, newest message last)
    # This helps the knowledge agent understand conversation flow
    messages = list(messages)
    messages.reverse()
    
    # Send the conversation context to the knowledge agent for intelligent retrieval
    retrieval_result = agent_client.retrieve(
        retrieval_request=KnowledgeAgentRetrievalRequest(
            # Convert messages to the format expected by the knowledge agent
            # Filter out system messages as they don't contain user queries
            messages=[
                KnowledgeAgentMessage(
                    role=msg["role"], 
                    content=[KnowledgeAgentMessageTextContent(text=msg.content[0].text)]
                ) 
                for msg in messages 
                if msg["role"] != "system"
            ],
            
            # Configure the target index and search parameters
            target_index_params=[
                KnowledgeAgentIndexParams(
                    index_name=index_name,      # Search in our Earth at night index
                    reranker_threshold=2.5      # Minimum relevance score for results
                )
            ]
        )
    )
    
    # Store the detailed retrieval results for later analysis
    # Associate with the most recent message in the conversation
    last_message = messages[-1]
    retrieval_results[last_message.id] = retrieval_result
    
    # Return the grounding response to the AI agent
    # This contains the search results formatted for the agent to use in its response
    return retrieval_result.response[0].content[0].text

# Create a function tool from our agentic retrieval function
# The AI agent can call this tool when it determines that information retrieval is needed
# Reference: https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/function-calling
functions = FunctionTool({ agentic_retrieval })

# Create a toolset and add our function tool
# Toolsets group related tools that an agent can use
toolset = ToolSet()
toolset.add(functions)

# Enable automatic function calling for the AI agent
# This allows the agent to automatically decide when to call our agentic_retrieval function
# The agent will analyze user queries and determine if search is needed
project_client.agents.enable_auto_function_calls(toolset)

In [39]:
# Test the knowledge agent configuration and retrieval functionality
# This cell performs a direct test of the retrieval system before full agent integration

# Get detailed information about our knowledge agent
agent_info = index_client.get_agent(agent_name)
# Display which indexes the agent can search
# This should show our "earth-at-night" index
print("Agent target indexes:", [ti.index_name for ti in agent_info.target_indexes])

# Create a test message to verify the retrieval system works
test_messages = [
    {"role": "user", "content": "What is the main topic of the NASA Earth at Night e-book?"}
]

# Perform a direct retrieval test using the knowledge agent client
# This bypasses the AI agent and tests the search functionality directly
retrieval_result = agent_client.retrieve(
    retrieval_request=KnowledgeAgentRetrievalRequest(
        # Convert test message to the format expected by the knowledge agent
        messages=[
            KnowledgeAgentMessage(
                role=msg["role"], 
                content=[KnowledgeAgentMessageTextContent(text=msg["content"])]
            ) 
            for msg in test_messages
        ],
        
        # Configure search parameters for the test
        target_index_params=[
            KnowledgeAgentIndexParams(
                index_name=index_name,      # Search our Earth at night index
                reranker_threshold=2.5      # Use same threshold as in production
            )
        ]
    )
)

# Display the raw retrieval response
# This shows what the knowledge agent found and how it formatted the results
# The response should contain relevant information about the NASA e-book
print("Retrieval response:", retrieval_result.response[0].content[0].text)

Agent target indexes: ['12-earth-at-night']
Retrieval response: [{"ref_id":0,"content":"interactions between society and natural processes. Beyond academic study, Earth nightlight measurements are being used to help save lives and property around the globe, by allowing accurate identification and monitoring of ongoing events like eruptions and fires even in remote locations, and by pinpointing and enabling quantitative tracking of regions of power outage and recovery following extreme weather events and geohazards in populated areas.\n\nEarth at Night tells—in the words of the women and men, the scientists and engineers who are actually designing the instruments and conducting the analyses of Earth's nocturnal illumination imagery—the story of satellite measurements of global light in the night. It shows how ever-increasing instrument capability has improved the sensitivity, accuracy, coverage, and resolution of the observations. Through striking illustrations and clear explanations, t

In [40]:
# Demonstrate where the agentic_retrieval function is stored
print("=== agentic_retrieval Function Storage Information ===")

# 1. The function exists in the current Python namespace (memory)
print(f"Function name: {agentic_retrieval.__name__}")
print(f"Function type: {type(agentic_retrieval)}")
print(f"Function location in memory: {id(agentic_retrieval)}")

# 2. Check if it's part of the FunctionTool
print(f"\nFunctionTool contains: {functions}")
print(f"FunctionTool type: {type(functions)}")

# 3. Check if it's registered with the agent's toolset
print(f"\nToolset object: {toolset}")
print("Toolset type:", type(toolset))

# 4. The function's docstring and signature
print(f"\nFunction signature: {agentic_retrieval.__doc__}")

# 5. Show where it's NOT stored (it's ephemeral)
print("\n=== What this function is NOT stored in: ===")
print("❌ Not in a database")
print("❌ Not in a file on disk") 
print("❌ Not persisted in Azure")
print("❌ Not saved between notebook sessions")
print("✅ Only exists in current Python runtime memory")
print("✅ Must be redefined if kernel restarts")

=== agentic_retrieval Function Storage Information ===
Function name: agentic_retrieval
Function type: <class 'function'>
Function location in memory: 1446108892032

FunctionTool contains: <azure.ai.agents.models._patch.FunctionTool object at 0x00000150AE1EA8D0>
FunctionTool type: <class 'azure.ai.agents.models._patch.FunctionTool'>

Toolset object: <azure.ai.agents.models._patch.ToolSet object at 0x00000150AE08A2C0>
Toolset type: <class 'azure.ai.agents.models._patch.ToolSet'>

Function signature: 
Agentic retrieval function that searches NASA e-book data about Earth at night.

This function serves as a "tool" that the AI agent can call when it needs to
retrieve information from the search index. The agent decides autonomously
when to use this tool based on the user's query.

Returns:
    str: JSON-formatted search results with reference IDs for citation

The function performs these steps:
1. Gets recent conversation history for context
2. Sends the conversation to the knowledge agent

## Start a chat with the agent

During the chat, you use the standard Azure AI agent tool calling APIs.  We send the message with questions, and the agent decides when to retrieve knowledge from your search index using agentic retrieval.

The remaining cells take a closer look at output and show how to add another turn to the conversation.

In [41]:
# Import classes for controlling agent tool selection
from azure.ai.agents.models import AgentsNamedToolChoice, AgentsNamedToolChoiceType, FunctionName

# Create a user message with complex questions about Earth at night phenomena
# These questions test the agent's ability to:
# - Handle multi-part queries
# - Understand scientific concepts
# - Find specific information in the knowledge base
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
)

# Execute the agent run with forced tool usage
# We explicitly force the agent to use our agentic_retrieval tool to ensure
# it searches for information rather than relying only on its training data
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,        # Use our conversation thread
    agent_id=agent.id,          # Use our created agent
    
    # Force the agent to use our agentic_retrieval function
    # This ensures the agent will search the index for information
    tool_choice=AgentsNamedToolChoice(
        type=AgentsNamedToolChoiceType.FUNCTION, 
        function=FunctionName(name="agentic_retrieval")
    ),
    
    toolset=toolset  # Provide access to our retrieval tool
)

# Check if the agent run completed successfully
# If it failed, we want to know why (could be model issues, tool problems, etc.)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")

# Get the agent's response to our question
# The agent should have used our retrieval tool to find relevant information
# and formatted it according to our instructions (with citation references)
output = project_client.agents.messages.get_last_message_text_by_role(
    thread_id=thread.id, 
    role="assistant"
).text.value

# Display the agent's response with improved formatting
# Replace periods with newlines for better readability of the detailed response
print("Agent response:", output.replace(".", "\n"))

Agent response: The suburban belts display larger December brightening than urban cores likely because these areas have more lighting installed that responds to seasonal and decorative changes, such as holiday lights, whereas absolute light levels remain higher in the urban cores due to constant and dense urban lighting infrastructure


Regarding Phoenix, the nighttime street grid is sharply visible from space because the metropolitan area is laid out along a regular and extensive grid of city blocks and streets
 This grid includes lighting from major transportation corridors, industrial and commercial facilities, and residential streets
 The lighting at intersections and along streets creates a distinct grid pattern visible from low-earth orbit vantage points like the ISS
 In contrast, large stretches of interstate between Midwestern cities remain comparatively dim likely because they have less urban development or street lighting coverage along those routes, and the lighting is less 

## Review retrieval activity and results

Each retrieval response from Azure AI Search includes the unified string (grounding data from search search results), the query plan, and  reference data showing which chunks of source document contributed content to the unified string.

In [42]:
# Import JSON for pretty-printing the retrieval analysis
import json

# Retrieve the detailed retrieval results for the message we just processed
# This gives us insight into what the knowledge agent actually did behind the scenes
retrieval_result = retrieval_results.get(message.id)

# Ensure we have retrieval results for this message
# This could fail if the agent didn't use our retrieval tool or if there was an error
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

# Display the retrieval activity log
# This shows the step-by-step process the knowledge agent went through:
# - Query analysis and understanding
# - Search strategy selection (keyword, vector, semantic, hybrid)
# - Query reformulation and expansion
# - Result ranking and filtering
print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))

# Display the retrieval results and references
# This shows:
# - Which documents were found and considered relevant
# - Relevance scores for each result
# - The specific chunks of text that contributed to the response
# - Reference IDs that should appear in the agent's response citations
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))

Retrieval activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1383,
    "output_tokens": 560
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "12-earth-at-night",
    "query": {
      "search": "Why do suburban areas show greater December brightening in light levels compared to urban cores despite higher absolute light levels downtown?"
    },
    "query_time": "2025-07-20T12:49:23.361Z",
    "count": 0,
    "elapsed_ms": 244
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "12-earth-at-night",
    "query": {
      "search": "Why is the Phoenix nighttime street grid so distinctly visible from space, while large stretches of interstate highways between Midwestern cities appear comparatively dim?"
    },
    "query_time": "2025-07-20T12:49:23.570Z",
    "count": 2,
    "elapsed_ms": 208
  },
  {
    "id": 3,
    "type": "AzureSearchSemanticRanker",
    "input_tokens": 49699
  }
]
Retrieval results
[
  {
    "

## Continue the conversation...

In [43]:
# Continue the conversation with a new question
# This demonstrates multi-turn conversation capabilities and context maintenance
message = project_client.agents.messages.create(
    thread_id=thread.id,  # Use the same thread to maintain conversation context
    role="user",
    content="How do I find lava at night? Use the retrieval tool to answer this question."
)
# Note: This question explicitly asks the agent to use the retrieval tool
# This tests whether the agent can find information about thermal detection
# and volcanic activity observation from space

# Execute another agent run with the same configuration
# The agent now has the context of the previous conversation
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,        # Same conversation thread for context
    agent_id=agent.id,          # Same agent instance
    
    # Again force the use of our retrieval tool
    # This ensures consistent behavior for demonstration purposes
    tool_choice=AgentsNamedToolChoice(
        type=AgentsNamedToolChoiceType.FUNCTION, 
        function=FunctionName(name="agentic_retrieval")
    ),
    
    toolset=toolset  # Same toolset with our retrieval function
)

# Check for execution errors
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")

# Get the agent's response to the lava detection question
# The response should incorporate both the new query and any relevant context
# from the previous conversation if applicable
output = project_client.agents.messages.get_last_message_text_by_role(
    thread_id=thread.id, 
    role="assistant"
).text.value

# Display the response with formatting for better readability
print("Agent response:", output.replace(".", "\n"))

Agent response: To find lava at night, you can look for the natural glow emitted by hot lava flows at active volcanoes
 This glow is visible from space and distinct from city lights because the lava emits bright red and orange lights
 Such glowing lava flows can be tracked and monitored using nighttime satellite imagery that detects the thermal and visible light emitted by the lava
 For example, eruptions like those of Mount Etna in Italy and Kilauea in Hawaii show bright spots of lava flow that stand out in nighttime images
 These observations can be enhanced by moonlight, which helps illuminate the surrounding area and allows for better visibility of the glowing lava [ref_id 1, 2, 3, 4]



## Review retrieval activity and results


In [44]:
# Analyze the retrieval results for the second question (about finding lava at night)
# This allows us to compare how the knowledge agent handled different types of queries

# Get the retrieval results for the lava detection question
retrieval_result = retrieval_results.get(message.id)

# Verify we have results for this message
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

# Display the retrieval activity for the lava detection query
# Compare this with the previous query to see how the agent:
# - Adapted its search strategy for a different topic
# - Used different keywords or semantic understanding
# - Potentially found different types of content
print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))

# Display the specific results and references found for lava detection
# This might include information about:
# - Thermal imaging techniques
# - Infrared detection methods
# - Volcanic monitoring from satellite imagery
# - Temperature signatures of geological activity
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))

Retrieval activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1622,
    "output_tokens": 129
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "12-earth-at-night",
    "query": {
      "search": "How do I find lava at night?"
    },
    "query_time": "2025-07-20T12:50:54.311Z",
    "count": 6,
    "elapsed_ms": 1721
  },
  {
    "id": 2,
    "type": "AzureSearchSemanticRanker",
    "input_tokens": 23060
  }
]
Retrieval results
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
    "activity_source": 1,
    "doc_key": "earth_at_night_508_page_44_verbalized"
  },
  {
    "type": "AzureSearchDoc",
    "id": "1",
    "activity_source": 1,
    "doc_key": "earth_at_night_508_page_65_verbalized"
  },
  {
    "type": "AzureSearchDoc",
    "id": "2",
    "activity_source": 1,
    "doc_key": "earth_at_night_508_page_64_verbalized"
  },
  {
    "type": "AzureSearchDoc",
    "id": "3",
    "activity_source": 1,
    "doc_key": "earth_at_night_50

## Clean up objects and resources

If you no longer need the resources, be sure to delete them from your Azure subscription.  You can also delete individual objects to start over.

### Delete the agent

In [ ]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.delete_agent(agent_name)
print(f"Knowledge agent '{agent_name}' deleted successfully")

### Delete the Index

In [ ]:
index_client = SearchIndexClient(endpoint=endpoint, credential=credential)
index_client.delete_index(index)
print(f"Index '{index_name}' deleted successfully")

### Delete the AI agent

In [ ]:
# Delete the AI agent from Azure AI Foundry to clean up resources

# Use the project_client to delete the agent by its ID
project_client.agents.delete_agent(agent.id)
print(f"AI agent '{agent.name}' deleted successfully")